## Using GRIB data

In [1]:
import earthkit.data as ekd
ekd.download_example_file("test6.grib")

In [2]:
ds = ekd.from_source("file", "test6.grib")

No GRIB data is actually loaded at this point.

### Iteration

In [3]:
for f in ds:
    print(f)

GribField(t,1000,20180801,1200,0,0)
GribField(u,1000,20180801,1200,0,0)
GribField(v,1000,20180801,1200,0,0)
GribField(t,850,20180801,1200,0,0)
GribField(u,850,20180801,1200,0,0)
GribField(v,850,20180801,1200,0,0)


### Inspecting the contents

In [4]:
len(ds)

6

In [5]:
ds.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,u,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
2,ecmf,v,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
3,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
4,ecmf,u,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
5,ecmf,v,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [6]:
ds.describe()

,,level,date,time,step,paramId,class,stream,type,experimentVersionNumber
shortName,typeOfLevel,,,,,,,,,
t,isobaricInhPa,"1000,850",20180801,1200,0,130,od,oper,an,0001
u,isobaricInhPa,"1000,850",20180801,1200,0,131,od,oper,an,0001
v,isobaricInhPa,"1000,850",20180801,1200,0,132,od,oper,an,0001


### Slicing

Standard Python slicing is available. It does not involve any loading/copying of GRIB data. 

In [7]:
g = ds[1]
g

GribField(u,1000,20180801,1200,0,0)

In [8]:
g = ds[1:3]
g.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,u,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,v,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll


In [9]:
g = ds[-1]
g

GribField(v,850,20180801,1200,0,0)

### Getting data values

#### Using values

In [10]:
v = ds[0].values
v.shape

(84,)

In [11]:
v[0:4]

array([272.56417847, 272.56417847, 272.56417847, 272.56417847])

In [12]:
v = ds.values
v.shape

(6, 84)

#### Using to_numpy()

In [13]:
v = ds[0].to_numpy()
print(v.shape)
print(ds[0].shape)

(7, 12)
(7, 12)


In [14]:
v = ds.to_numpy()
v.shape

(6, 7, 12)

### Metadata

In [15]:
ds[0].metadata("typeOfLevel")

'isobaricInhPa'

In [16]:
ds[0:2].metadata(["level", "paramId"])

[[1000, 130], [1000, 131]]

In [17]:
ds.metadata("level")

[1000, 1000, 1000, 850, 850, 850]

Key qualifiers can be used to prescribe the required metadata type:

In [18]:
ds[0].metadata(["centre", "centre", "centre"], astype=(None, int, str))

['ecmf', 98, 'ecmf']

In [19]:
md = ds[0].metadata()
md

In [20]:
md["level"]

1000

### Selection

In [21]:
g = ds.sel(shortName=["u", "v"], level=850)
g.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,u,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
1,ecmf,v,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [22]:
g = ds.sel(param="t")
g.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


### Xarray

In [23]:
xds = ds.to_xarray(engine="cfgrib")
xds

<xarray.Dataset> Size: 2kB
Dimensions:        (number: 1, time: 1, step: 1, isobaricInhPa: 2, latitude: 7,
                    longitude: 12)
Coordinates:
  * number         (number) int64 8B 0
  * time           (time) datetime64[ns] 8B 2018-08-01T12:00:00
  * step           (step) timedelta64[ns] 8B 00:00:00
  * isobaricInhPa  (isobaricInhPa) float64 16B 1e+03 850.0
  * latitude       (latitude) float64 56B 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * longitude      (longitude) float64 96B 0.0 30.0 60.0 ... 270.0 300.0 330.0
    valid_time     (time, step) datetime64[ns] 8B ...
Data variables:
    t              (number, time, step, isobaricInhPa, latitude, longitude) float32 672B ...
    u              (number, time, step, isobaricInhPa, latitude, longitude) float32 672B ...
    v              (number, time, step, isobaricInhPa, latitude, longitude) float32 672B ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-03-10T15:18 GRIB to CDM+CF via cfgrib-0.9.1...

In [24]:
xds = ds.to_xarray(engine="cfgrib", xarray_open_dataset_kwargs={"backend_kwargs": {"ignore_keys": ["number"]}})
xds

<xarray.Dataset> Size: 2kB
Dimensions:        (time: 1, step: 1, isobaricInhPa: 2, latitude: 7,
                    longitude: 12)
Coordinates:
  * time           (time) datetime64[ns] 8B 2018-08-01T12:00:00
  * step           (step) timedelta64[ns] 8B 00:00:00
  * isobaricInhPa  (isobaricInhPa) float64 16B 1e+03 850.0
  * latitude       (latitude) float64 56B 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * longitude      (longitude) float64 96B 0.0 30.0 60.0 ... 270.0 300.0 330.0
    valid_time     (time, step) datetime64[ns] 8B ...
Data variables:
    t              (time, step, isobaricInhPa, latitude, longitude) float32 672B ...
    u              (time, step, isobaricInhPa, latitude, longitude) float32 672B ...
    v              (time, step, isobaricInhPa, latitude, longitude) float32 672B ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-03-10T15:18 GRIB to CDM+CF via cfgrib-0.9.1...